In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import numpy
import scipy
import pylab as plt
import math
#
import sys
import os
import vtk
import h5py
#
import csv
import xml
import xml.etree.ElementTree as ET


In [11]:

data_path = os.path.join(os.path.split(os.getcwd())[0], 'vtkOutput')
print('** data_path: ', data_path)




** data_path:  /scratch/users/myoder96/haycat_geos/vtkOutput


# GEOS output layout
The directory structure is something like,
```
$ tree -L 5
.
└── vtkOutput
    ├── 000000
    │   └── mesh
    │       └── Level0
    │           ├── carbonate_cement
    │           ├── coarse_sandstone
    │           ├── composite_high
    │           ├── composite_low
    │           ├── composite_mid
    │           ├── wellRegionCRC3
    │           └── wellRegionCRC8
    ├── 000000.vtm
    ├── 000007
    │   └── mesh
    │       └── Level0
    │           ├── carbonate_cement
    │           ├── coarse_sandstone
    │           ├── composite_high
    │           ├── composite_low
    │           ├── composite_mid
    │           ├── wellRegionCRC3
    │           └── wellRegionCRC8
    ├── 000007.vtm
```

Where under each Level-5 entry (carbonate_cement, coarse_sandstone, ...) are N XML files, `rank_{k_mpi}.vtu`, where `k_mpi` are the MPI rank.

The most obvious TODO is to consolidate the MPI rank collections into a single file. In most cases, this will reduce the iNode count by ~100. Then, it is pretty straight forward (I think?) to stack the outputs as a time series.

There is a lot of redundant information. For example, at least the way this GEOS output is produced, the `time` index is repeated in each MPI rank, at least in this implementation, `mesh` is the only output, so the only member of the `{time_step}` directory, and then we only have the one `Level0`. So really everything can be aggregated up to the `{time_step}` level.

Note that the directory structure is described in the `{timestep}.vtm` XML file, eg:

```
<?xml version="1.0"?>
<VTKFile type="vtkMultiBlockDataSet" version="1.0">
        <vtkMultiBlockDataSet>
                <Block name="mesh">
                        <Block name="Level0">
                                <Block name="CellElementRegion">
                                        <Block name="coarse_sandstone">
                                                <DataSet name="rank_000" file="000022/mesh/Level0/coarse_sandstone/rank_000.vtu" />
                                                <DataSet name="rank_001" file="000022/mesh/Level0/coarse_sandstone/rank_001.vtu" />
                                                <DataSet name="rank_002" file="000022/mesh/Level0/coarse_sandstone/rank_002.vtu" />
                                                <DataSet name="rank_003" file="000022/mesh/Level0/coarse_sandstone/rank_003.vtu" />
```

Then, of course, the data must be extracted from each `{rank}..vtu` file, the process for which is not obvious. So we will start with just a script to describe the structure to aggregate. We will base this aggregation process on the `{timestep}.vtm` XML file, rather than an external knowledge of the directory structure. 


In [68]:
# tree of the timeseries. Start with one file.
# as expected, at the root level, we have only the single 'vtkMultBlockDataSet' entry.
#
tree_ts = ET.parse(os.path.join(data_path, '000000.vtm'))
root_ts = tree_ts.getroot()
#
# I think we need to manually handle the recursion.
for k,child in enumerate(tree_ts.getroot()):
    if k>0: break
        
    print(f'** item[{k}]: {child}: {child.tag} :: {child.attrib}')
    for k_mesh, mesh in enumerate(child): 
        print(f'** [{k, k_mesh}]: {child.tag}:: {mesh.attrib}')
        for k_level, level in enumerate(mesh):
            print(f'** [{k, k_mesh, k_level}]: {mesh.attrib["name"]}:: {level.attrib["name"]}')
            for k_region, region in enumerate(level):
                print(f'**[{k, k_mesh, k_level, k_level, k_region}]: {level.attrib["name"]}:: {region.attrib["name"]}\n')
                for k_element, element in enumerate(region):
                    #
                    # Here is where would aggregate the ranks
                    for k_rank, rank in enumerate(element):
                        # read the file; aggregate...
                        # get a list of data sets. Assume they'll all be in the first file
                        if k_rank>0:
                            pass
                        else:
                            #
                            print(f'\n*** rank file [{k_rank}]: {rank.attrib["file"]}')
                            rank_file = rank.attrib['file']
                            rank_tree = ET.parse(os.path.join(data_path, rank_file))
                            #
                            rank_root = rank_tree.getroot()
                            #print('** rank_root:', rank_root)
                            for grid in rank_root:
                                # should be only one of these? But maybe we should prepare for the general case?
                                print(f'** grid: {grid}')
                                # There should be only one FieldData, pluse one "Piece"?
                                for k_fd, fd in enumerate(grid):
                                    print(f'**  fielddata[{k_fd}]: {fd.tag}: {fd.get("Name", "(no-name)")}')
                                    # in the "FieldData" level (FieldData, Piece)
                                    for da_pd in fd:
                                        print(f'**       DataCategory: {da_pd.tag}, {da_pd.attrib.get("Name","na")}')
                                        #
                                        if da_pd.tag == 'DataArray':
                                            print(f'**       is DataArray: {da_pd.attrib["Name"]}')
                                            #print('*** ', numpy.array(da_pd.text))
                                        if fd.tag == 'Piece':
                                            # Piece of the sim?
                                            # Data sets should be here:
                                            for da in da_pd:
                                                print(f'** DataArray: {da.tag}: {fd.tag}::{da_pd.tag}::{da.attrib["Name"]}')
                                            pass
                            
                    print(f'** [{k, k_mesh, k_level, k_element}]: {region.attrib["name"]}:: {element.attrib["name"]}::\
                          {k_rank+1} rank files')
                                          
                                          

** item[0]: <Element 'vtkMultiBlockDataSet' at 0x7f566de8e4a0>: vtkMultiBlockDataSet :: {}
** [(0, 0)]: vtkMultiBlockDataSet:: {'name': 'mesh'}
** [(0, 0, 0)]: mesh:: Level0
**[(0, 0, 0, 0, 0)]: Level0:: CellElementRegion


*** rank file [0]: 000000/mesh/Level0/coarse_sandstone/rank_000.vtu
** grid: <Element 'UnstructuredGrid' at 0x7f566de8f4f0>
**  fielddata[0]: FieldData: (no-name)
**       DataCategory: DataArray, TIME
**       is DataArray: TIME
**  fielddata[1]: Piece: (no-name)
**       DataCategory: PointData, na
** DataArray: DataArray: Piece::PointData::localToGlobalMap
** DataArray: DataArray: Piece::PointData::ghostRank
**       DataCategory: CellData, na
** DataArray: DataArray: Piece::CellData::relperm_coarse_sandstone_hyst_phaseRelPerm
** DataArray: DataArray: Piece::CellData::rockPorosity_referencePorosity
** DataArray: DataArray: Piece::CellData::rockPorosity_porosity
** DataArray: DataArray: Piece::CellData::fluid_totalDensity
** DataArray: DataArray: Piece::CellData::

In [13]:
# can we do that with a generalized recursion? will it remember the variables in the correct scope?
#
k_recurse = 0
k_r_max = 5
#
items = tree_ts.getroot()
while k_recurse < k_r_max:
    for k, elem in enumerate(items):
        print(f'item [{k}]: {items.attrib}: {elem.attrib["name"]}')
        items = elem
        k_recurse +=1
        #
    

KeyError: 'name'